In [1]:
###Upload my -eve files here

In [2]:
import numpy as np
import pandas as pd
import copy
import collections

In [3]:
angels=pd.read_csv("/Users/devon.sinha/Desktop/Math 390/2020EVE/2020ANA.csv")
angels=angels[angels.id=="play"]

angels.columns=["id", "inning", "AorH", "toDel", "del", "Delete", "result"]
mTrout=angels
mTrout=mTrout.drop(columns=["id", "del", "Delete"])
mTrout=mTrout[mTrout.toDel=="troum001"]


angels=angels.drop(columns=["id","toDel", "del", "Delete"])

print(angels.head())

angels=angels.reset_index()
angels=angels.drop(columns=["index"])

print(angels.head())

global df

df=angels



   inning AorH  result
52      1    0       K
53      1    0  53/G56
54      1    0  S8/G4M
55      1    0  8/F8XD
56      1    1       W
  inning AorH  result
0      1    0       K
1      1    0  53/G56
2      1    0  S8/G4M
3      1    0  8/F8XD
4      1    1       W


In [4]:
###make functions here
globalDict={}
def initialize_state():
    global df
    ##need to make sure Df's size actually gets modified in functions IT currently is not!!
    while np.size(df)>0:
        calcStates(globalDict)
    
    
def updateDict(globalDict):
    global current_state
    #print("UpdtateDict",totalruns)
    if(tuple(current_state) not in globalDict):
        globalDict[tuple(current_state)]=[0, 0]
        
    globalDict[tuple(current_state)][0]+=1
    globalDict[tuple(current_state)][1]-=(totalruns)
    #print("endUpdate",globalDict[tuple(current_state)][1])
    
    
def calcStates(globalDict):
    global current_state
    global totalruns
    global df

    
    totalruns=0
    current_inning=df.iloc[0][0]
    current_AorH=df.iloc[0][1]
    current_state=[0,0,0,0]
    states=[]
    ###while in the same half inning
    ##need to edit so it only runs for a half inning
    while(np.size(df)>0 and current_inning==df.iloc[0][0] and current_AorH==df.iloc[0][1]):
        ###need to process and delete rows here
        play=df.iloc[0][2]
        df=df.drop(labels=0, axis=0)
        df=df.reset_index()
        df=df.drop(columns=["index"])
        spt=play.split(".")
        if(len(spt)>1):
            advanced= spt[1]
            advanceRunners(advanced)
        hit=play.split(".")[0]
        processHit(hit)
        if current_state[0]<3:
            updateDict(globalDict)
            states.append(copy.copy(current_state))
    finalUpdate(states)

def finalUpdate(states): 
    for elem in states:
        globalDict[tuple(elem)][1]+=totalruns
    

def processHit(hit):
    global current_state
    global totalruns
    firstLetter=hit[0]
    doublePlay="DP" in hit or "dp" in hit
    out=firstLetter not in ["S","D", "T", "W", "H", "E", "N", "P"]
    notOut=firstLetter in ["S", "D", "T", "W", "H"]
    if out:
        if doublePlay:
            current_state[0]+=2
        else:
            current_state[0]+=1
        if current_state[0]>3:
            current_state[0]=3
    if notOut:
        if firstLetter=="S" or firstLetter=="W":
            current_state[1]=1
        if firstLetter=="D":
            current_state[2]=1
        if firstLetter=="T":
            current_state[3]=1
        if firstLetter=="H":
            totalruns+=1
    

##advanceRunners takes care of any advancements
def advanceRunners(advanced):
    global current_state
    global totalruns
    if "-" in advanced:
        advanced=advanced.replace("H", "4")
        advancements=advanced.split(";")
        for elem in advancements:
            if(elem[0] in ["1","2","3"]):
                current_state[int(elem[0])]=0
            if int(elem[2])==4:
                totalruns+=1
            else:
                current_state[int(elem[2])]=1

            
def calculateRuns(globalDict):
    averageRunsPerState={}
    for key in globalDict:
        averageRunsPerState[key]=globalDict[key][1]/globalDict[key][0]
     
    od = collections.OrderedDict(sorted(averageRunsPerState.items()))
    #for k, v in averageRunsPerState.items():
        #print(k, v)
                       

    for k, v in od.items(): 
        print(k, v)

    

In [5]:
### repeat CalcStates until DF is empty



In [6]:
initialize_state()


In [7]:
print("EXPECTED AMOUNT OF RUNS IN EACH STATE FOR THE 2020 ANGELS")
calculateRuns(globalDict)

EXPECTED AMOUNT OF RUNS IN EACH STATE FOR THE 2020 ANGELS
(0, 0, 0, 0) 0.6791044776119403
(0, 0, 0, 1) 1.1428571428571428
(0, 0, 1, 0) 1.1458333333333333
(0, 0, 1, 1) 2.1666666666666665
(0, 1, 0, 0) 1.0677966101694916
(0, 1, 0, 1) 1.588235294117647
(0, 1, 1, 0) 1.7868852459016393
(0, 1, 1, 1) 2.411764705882353
(1, 0, 0, 0) 0.31313131313131315
(1, 0, 0, 1) 0.8888888888888888
(1, 0, 1, 0) 0.84
(1, 0, 1, 1) 1.2380952380952381
(1, 1, 0, 0) 0.6116504854368932
(1, 1, 0, 1) 1.3170731707317074
(1, 1, 1, 0) 0.9142857142857143
(1, 1, 1, 1) 2.210526315789474
(2, 0, 0, 0) 0.14556962025316456
(2, 0, 0, 1) 0.47619047619047616
(2, 0, 1, 0) 0.32857142857142857
(2, 0, 1, 1) 0.6
(2, 1, 0, 0) 0.30869565217391304
(2, 1, 0, 1) 0.6451612903225806
(2, 1, 1, 0) 0.5425531914893617
(2, 1, 1, 1) 1.372093023255814


### Comparing Expected Runs between "worst" and "best" teams in 2020

For analysis, I want to compare the expected amount of runs at each state between the "best" and "worst" team performacne wise in the MLB in 2020. I am comparing the Dodgers and the Pirates to analyze which states in which the Dodgers outperfrom the Pirates

In [8]:
dodgers=pd.read_csv("/Users/devon.sinha/Desktop/Math 390/2020EVE/2020LAN.csv")
#print(dodgers.head())
dodgers=dodgers[dodgers.id=="play"]

dodgers.columns=["id", "inning", "AorH", "toDel", "del", "Delete", "result"]



dodgers=dodgers.drop(columns=["id","toDel", "del", "Delete"])

print(dodgers.head())

dodgers=dodgers.reset_index()
dodgers=dodgers.drop(columns=["index"])

print(dodgers.head())

#global df

df=dodgers
globalDict={}



   inning AorH       result
52      1    0    E6/G+.B-1
53      1    0          9/F
54      1    0  S5/G.1X2(5)
55      1    0     S6/G.1-2
56      1    0        13/G-
  inning AorH       result
0      1    0    E6/G+.B-1
1      1    0          9/F
2      1    0  S5/G.1X2(5)
3      1    0     S6/G.1-2
4      1    0        13/G-


In [9]:
initialize_state()

In [10]:
print("EXPECTED AMOUNT OF RUNS IN EACH STATE FOR THE 2020 Dodgers")
calculateRuns(globalDict)

EXPECTED AMOUNT OF RUNS IN EACH STATE FOR THE 2020 Dodgers
(0, 0, 0, 0) 0.7103448275862069
(0, 0, 0, 1) 4.0
(0, 0, 1, 0) 1.6153846153846154
(0, 0, 1, 1) 1.9
(0, 1, 0, 0) 0.7777777777777778
(0, 1, 0, 1) 1.7692307692307692
(0, 1, 1, 0) 1.4857142857142858
(0, 1, 1, 1) 1.9473684210526316
(1, 0, 0, 0) 0.3117505995203837
(1, 0, 0, 1) 0.7142857142857143
(1, 0, 1, 0) 0.5957446808510638
(1, 0, 1, 1) 1.8181818181818181
(1, 1, 0, 0) 0.5674157303370787
(1, 1, 0, 1) 0.7777777777777778
(1, 1, 1, 0) 0.7049180327868853
(1, 1, 1, 1) 0.9090909090909091
(2, 0, 0, 0) 0.11384615384615385
(2, 0, 0, 1) 0.5454545454545454
(2, 0, 1, 0) 0.3220338983050847
(2, 0, 1, 1) 1.1875
(2, 1, 0, 0) 0.2672811059907834
(2, 1, 0, 1) 0.4727272727272727
(2, 1, 1, 0) 0.5466666666666666
(2, 1, 1, 1) 0.7727272727272727


In [11]:
pirates=pd.read_csv("/Users/devon.sinha/Desktop/Math 390/2020EVE/2020PIT.csv")
#print(dodgers.head())
pirates=pirates[pirates.id=="play"]

pirates.columns=["id", "inning", "AorH", "toDel", "del", "Delete", "result"]



pirates=pirates.drop(columns=["id","toDel", "del", "Delete"])

print(pirates.head())

pirates=pirates.reset_index()
pirates=pirates.drop(columns=["index"])

print(pirates.head())

#global df

df=pirates
globalDict={}



   inning AorH   result
52      1    0    53/G5
53      1    0   S8/G6M
54      1    0        K
55      1    0  13/G1S-
56      1    1    31/G3
  inning AorH   result
0      1    0    53/G5
1      1    0   S8/G6M
2      1    0        K
3      1    0  13/G1S-
4      1    1    31/G3


In [12]:
initialize_state()

In [13]:
print("EXPECTED AMOUNT OF RUNS IN EACH STATE FOR THE 2020 Pirates")
calculateRuns(globalDict)

EXPECTED AMOUNT OF RUNS IN EACH STATE FOR THE 2020 Pirates
(0, 0, 0, 0) 0.5909090909090909
(0, 0, 0, 1) 1.0
(0, 0, 1, 0) 1.5869565217391304
(0, 0, 1, 1) 3.1818181818181817
(0, 1, 0, 0) 0.8733333333333333
(0, 1, 0, 1) 1.5294117647058822
(0, 1, 1, 0) 2.111111111111111
(0, 1, 1, 1) 2.8076923076923075
(1, 0, 0, 0) 0.30295566502463056
(1, 0, 0, 1) 1.1666666666666667
(1, 0, 1, 0) 0.6181818181818182
(1, 0, 1, 1) 1.4
(1, 1, 0, 0) 0.5333333333333333
(1, 1, 0, 1) 0.625
(1, 1, 1, 0) 1.0819672131147542
(1, 1, 1, 1) 1.7307692307692308
(2, 0, 0, 0) 0.12025316455696203
(2, 0, 0, 1) 0.3157894736842105
(2, 0, 1, 0) 0.2602739726027397
(2, 0, 1, 1) 0.7
(2, 1, 0, 0) 0.33185840707964603
(2, 1, 0, 1) 0.4909090909090909
(2, 1, 1, 0) 0.5955056179775281
(2, 1, 1, 1) 0.8181818181818182
